In [1]:
import socket
import pickle
import tensorflow as tf
import time

In [2]:
import numpy as np 
import pandas as pd 

import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
with open('text_input.txt', encoding='utf-8') as file:
    data = file.read()

In [4]:
soup=BeautifulSoup(data,'html.parser')#remove html tag using beautifulsoup
non_html_text=soup.get_text()
    
#remove unwanted charectors and symbols
text=re.sub('[^a-zA-Z0-9\s]',' ',non_html_text)
text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '')

#remove extra spaces
z = []
for i in text.split():
    if i not in z:
        z.append(i)  
text = ' '.join(z)

#tokenize text
    
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
sequence=tokenizer.texts_to_sequences([text])[0]

sequence[:15]

[27, 28, 29, 1, 30, 31, 32, 33, 2, 34, 35, 2, 3, 4, 5]

In [5]:
ip_dim=len(tokenizer.word_index)+1
ip_dim

695

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
model=Sequential()
model.add(Embedding(ip_dim,10,input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(ip_dim,activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             6950      
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 695)               695695    
                                                                 
Total params: 13751645 (52.46 MB)
Trainable params: 13751645 (52.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
model.save('server.keras')

In [9]:
'''
def federated_averaging(global_weights, client_weights_list):
    num_clients = len(client_weights_list)
    averaged_weights = global_weights.copy()
    
    for i in range(len(averaged_weights)):
        # Initialize with the global weights
        averaged_weights[i] = global_weights[i]
        
        # Aggregate model updates from all clients
        for client_weights in client_weights_list:
            averaged_weights[i] += client_weights[i] / num_clients
    
    return averaged_weights
'''

'\ndef federated_averaging(global_weights, client_weights_list):\n    num_clients = len(client_weights_list)\n    averaged_weights = global_weights.copy()\n    \n    for i in range(len(averaged_weights)):\n        # Initialize with the global weights\n        averaged_weights[i] = global_weights[i]\n        \n        # Aggregate model updates from all clients\n        for client_weights in client_weights_list:\n            averaged_weights[i] += client_weights[i] / num_clients\n    \n    return averaged_weights\n'

In [10]:
'''
# Define the server's IP address and port
server_ip = '192.168.94.137'  # Replace with your server's local IP address
server_port = 12347

# Create a server socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the server socket to the IP address and port
server_socket.bind((server_ip, server_port))

# Listen for incoming connections
server_socket.listen(3)

print(f"Server is listening for incoming connections on {server_ip}:{server_port}...")


# Accept connections from multiple clients
# Simulate federated learning rounds
num_rounds = 6
num_clients= 3
    
# Receive model weights from each client
client_weights_list = []

for _ in range(num_clients):
    client_socket, client_address = server_socket.accept()
    print(f"Accepted connection from {client_address}")
    
    client_weights_bytes = b''
    for round_num in range(num_rounds):
        while True:
            data = client_socket.recv(13246848)  # Adjust buffer size as needed
            if not data:
                break
            client_weights_bytes += data
            

        if not client_weights_bytes:
            print(f"No data received from a client{_ + 1}, in round {round_num}.")
            continue

        try:
            client_model_weights = pickle.loads(client_weights_bytes)
            client_weights_list.append(client_model_weights)
            
        except pickle.UnpicklingError as e:
            print(f"Error while unpickling data from a client: {e}")
            
    client_socket.close()
'''

'\n# Define the server\'s IP address and port\nserver_ip = \'192.168.94.137\'  # Replace with your server\'s local IP address\nserver_port = 12347\n\n# Create a server socket\nserver_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n\n# Bind the server socket to the IP address and port\nserver_socket.bind((server_ip, server_port))\n\n# Listen for incoming connections\nserver_socket.listen(3)\n\nprint(f"Server is listening for incoming connections on {server_ip}:{server_port}...")\n\n\n# Accept connections from multiple clients\n# Simulate federated learning rounds\nnum_rounds = 6\nnum_clients= 3\n    \n# Receive model weights from each client\nclient_weights_list = []\n\nfor _ in range(num_clients):\n    client_socket, client_address = server_socket.accept()\n    print(f"Accepted connection from {client_address}")\n    \n    client_weights_bytes = b\'\'\n    for round_num in range(num_rounds):\n        while True:\n            data = client_socket.recv(13246848)  # Adjust buf

In [11]:
'''
# Aggregate model updates using federated averaging
if client_weights_list:
    global_weights = federated_averaging(model.get_weights(), client_weights_list)
    model.set_weights(global_weights)
    print(f"Updated global model with data from {num_clients} clients in round {len(client_weights_list)}.")

        
# The global model now contains the federated learning result
model.save_weights('model_weights.h5')

# Close the server socket
server_socket.close()
'''

'\n# Aggregate model updates using federated averaging\nif client_weights_list:\n    global_weights = federated_averaging(model.get_weights(), client_weights_list)\n    model.set_weights(global_weights)\n    print(f"Updated global model with data from {num_clients} clients in round {len(client_weights_list)}.")\n\n        \n# The global model now contains the federated learning result\nmodel.save_weights(\'model_weights.h5\')\n\n# Close the server socket\nserver_socket.close()\n'

In [12]:
'''
# For preprocessing input text for test
def preprocess(text):
    soup1=BeautifulSoup(text,'html.parser')#remove html tag using beautifulsoup
    non_html_txt=soup.get_text()
    
    #remove unwanted charectors and symbols
    text=re.sub('[^a-zA-Z0-9\s]',' ',non_html_txt)
    text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '')

    #remove extra spaces
    z = []
    for i in text.split():
        if i not in z:
            z.append(i)  
    text = ' '.join(z)
    return text
'''

"\n# For preprocessing input text for test\ndef preprocess(text):\n    soup1=BeautifulSoup(text,'html.parser')#remove html tag using beautifulsoup\n    non_html_txt=soup.get_text()\n    \n    #remove unwanted charectors and symbols\n    text=re.sub('[^a-zA-Z0-9\\s]',' ',non_html_txt)\n    text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '')\n\n    #remove extra spaces\n    z = []\n    for i in text.split():\n        if i not in z:\n            z.append(i)  \n    text = ' '.join(z)\n    return text\n"

In [13]:
'''
# It gives single predicted words
def predict_nxt_word(txt):

    tok_text = tokenizer.texts_to_sequences([ txt])
    #print(tok_text)
    preds = model.predict(np.array(tok_text), verbose=0)[0]
    next_word = tokenizer.sequences_to_texts([[np.argmax(preds)]])[0]
    return next_word
'''

'\n# It gives single predicted words\ndef predict_nxt_word(txt):\n\n    tok_text = tokenizer.texts_to_sequences([ txt])\n    #print(tok_text)\n    preds = model.predict(np.array(tok_text), verbose=0)[0]\n    next_word = tokenizer.sequences_to_texts([[np.argmax(preds)]])[0]\n    return next_word\n'

In [14]:
'''
txt='Can I'
txt=preprocess(txt)
txt=txt.split(' ')
txt=txt[-3:]
# print(txt)
nxt=predict_nxt_word(txt)
print(nxt)
'''

"\ntxt='Can I'\ntxt=preprocess(txt)\ntxt=txt.split(' ')\ntxt=txt[-3:]\n# print(txt)\nnxt=predict_nxt_word(txt)\nprint(nxt)\n"

In [15]:
'''
#formula caluclation
data_size = 79481088  # Bytes
num_iterations = 6

buffer_size = data_size // num_iterations

print(buffer_size)
'''

'\n#formula caluclation\ndata_size = 79481088  # Bytes\nnum_iterations = 6\n\nbuffer_size = data_size // num_iterations\n\nprint(buffer_size)\n'